In [1]:
import keras
from keras.models import Model
import pandas as pd
from keras import regularizers
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD, RMSprop, Adam
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
import json
import gensim
import glob
from keras.preprocessing import image
import numpy as np
import re
from keras import utils
import matplotlib.pyplot as plt
%matplotlib inline

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#Load Google's pre-trained Word2Vec model.
from gensim.models import KeyedVectors
word_vectors = KeyedVectors.load_word2vec_format('google_w2vec.bin', binary=True)

In [3]:
vecsize = word_vectors.vector_size
print('Vocabulary Size:', len(word_vectors.vocab))
print(type(word_vectors.vocab))
print('Vector Size:', vecsize)

Vocabulary Size: 3000000
<class 'dict'>
Vector Size: 300


In [4]:
#Define File-Path to Users Folder
folder_list = ['#foodporn',"#nightlife","#cosmetics","#rockclimbing"]
image_path = "/Users/kmotwani/Desktop/Me/Education/Courses/Capstone Project/Insta Images/"

In [5]:
#Helper Function to get images from path
def get_df(path, list_input, thresh):
    final_list = []
    for ind, i in enumerate(list_input):
        temp_path = path + i
        print(i,"\n")
        count = 0
        for j in glob.glob(temp_path + '/*.jpg'):
            temp_dict = {}
            file_name = j.replace(temp_path,'')[1:]
            img = image.load_img(j, target_size=(128, 128))
            try:
                with open(temp_path+"/"+file_name[:-4]+'.txt', encoding="utf-8") as f:
                    content = f.readlines()
                    caption = ' '.join([x.strip() for x in content])
            except FileNotFoundError:
                continue
            temp_dict['File'], temp_dict['Response'] = file_name, int(ind)
            temp_dict['Image'], temp_dict['Caption'] = np.array(img), caption
            final_list.append(temp_dict)
            count += 1
            if count==thresh:
                break
    return pd.DataFrame(final_list) 


#Get Images from User List and Path
df = get_df(image_path, folder_list, 9000)
print("Number of images loaded:", len(df))
display(df.head())

#foodporn 

#nightlife 

#cosmetics 

#rockclimbing 

Number of images loaded: 32595


,Caption,File,Image,Response
0,...Siempre quise ser grande para poder comerme...,2018-04-08_21-17-49_UTC.jpg,"[[[82, 68, 59], [88, 85, 96], [128, 104, 80], ...",0
1,Lonche de Panela By @mojito.restobar . . . #va...,2018-04-08_21-36-39_UTC.jpg,"[[[165, 158, 199], [162, 155, 196], [167, 159,...",0
2,Brunching with friends is the best way to spen...,2018-04-08_20-19-50_UTC.jpg,"[[[61, 51, 49], [48, 37, 31], [153, 149, 166],...",0
3,Obsessed with their little stoves. Ordered the...,2018-02-02_15-27-17_UTC.jpg,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",0
4,"Hoje para o jantar foram dois ovos, em cima de...",2018-04-08_21-36-29_UTC.jpg,"[[[147, 163, 160], [152, 163, 155], [131, 101,...",0


In [6]:
#Remove Hashtags
df['Caption'] = df['Caption'].str.replace('#foodporn','', case=False)
df['Caption'] = df['Caption'].str.replace('#rockclimbing','', case=False)
df['Caption'] = df['Caption'].str.replace('#nightlife','', case=False)
df['Caption'] = df['Caption'].str.replace('#cosmetics','', case=False)
display(df.head())

,Caption,File,Image,Response
0,...Siempre quise ser grande para poder comerme...,2018-04-08_21-17-49_UTC.jpg,"[[[82, 68, 59], [88, 85, 96], [128, 104, 80], ...",0
1,Lonche de Panela By @mojito.restobar . . . #va...,2018-04-08_21-36-39_UTC.jpg,"[[[165, 158, 199], [162, 155, 196], [167, 159,...",0
2,Brunching with friends is the best way to spen...,2018-04-08_20-19-50_UTC.jpg,"[[[61, 51, 49], [48, 37, 31], [153, 149, 166],...",0
3,Obsessed with their little stoves. Ordered the...,2018-02-02_15-27-17_UTC.jpg,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",0
4,"Hoje para o jantar foram dois ovos, em cima de...",2018-04-08_21-36-29_UTC.jpg,"[[[147, 163, 160], [152, 163, 155], [131, 101,...",0


In [7]:
#Split data into Train and Test set
use_df = df.sample(frac=1).reset_index(drop=True)
display(use_df.head())
np.random.seed(9001)
msk = np.random.rand(len(use_df)) < 0.7
total_data_train = use_df[msk]
total_data_test = use_df[~msk]

,Caption,File,Image,Response
0,"Training Insight 1! The slightly obvious, but ...",2018-04-07_18-00-21_UTC.jpg,"[[[141, 129, 115], [136, 112, 110], [108, 86, ...",3
1,#Repost from @tyan_bf ・・・ Dijual botol bekas k...,2018-04-08_11-06-47_UTC.jpg,"[[[59, 103, 148], [56, 100, 145], [54, 98, 143...",1
2,Le fond du coeur est plus loin que le bout du ...,2017-07-08_07-04-50_UTC.jpg,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",3
3,Testimoni produk Drk yg menjadi kegilaan ramai...,2018-04-08_16-38-04_UTC.jpg,"[[[211, 211, 211], [211, 211, 211], [211, 211,...",2
4,. 一緒に過ごした時間 楽しい会話 まるで友達のように . そんな時間の積み重ねが 大人の時...,2018-04-08_12-17-45_UTC.jpg,"[[[234, 247, 255], [234, 247, 255], [234, 247,...",2


In [8]:
#Helper function to get text vector
def get_vector(x, limit):
    sequence, count = np.zeros((limit, 300), dtype=float), 0
    x = re.sub(r'[^\w\s]','',x)
    for word in x.split():
        if word in word_vectors.vocab:
            if count<limit: 
                sequence[count] = word_vectors.get_vector(word)
        count += 1
    return sequence

#Define Train and Test Dataframes
x_train, x_test = [], []
total_data_train['Caption'].astype(str).apply(lambda x: x_train.append(get_vector(x, 100)))
total_data_test['Caption'].astype(str).apply(lambda x: x_test.append(get_vector(x, 100)))
y_train = utils.to_categorical(total_data_train['Response'].as_matrix(), num_classes=4)
y_test = utils.to_categorical(total_data_test['Response'].as_matrix(), num_classes=4)

In [9]:
#Change X,Y to Numpy Arrays
x_train = np.array(x_train)
x_test = np.array(x_test)

In [15]:
#Helper function to create CNN Model for Image Classification
def createModel(size, classes):
    model = Sequential()
    model.add(keras.layers.LSTM(size, input_shape=(100, 300), return_sequences=False))
    model.add(Dense(classes, activation='tanh'))  
    model.summary()
    return model

#Helper function to run model and save intermediate weights
def run_model(model, x_train, y_train, x_test, y_test, batch_size, epochs):
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    filepath="KRM_LSTM_New_weights-{epoch:02d}-{val_acc:.2f}.hdf5"
    check = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max', period = 5)
    callbacks_list = [check]
    history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, 
                       validation_data=(x_test, y_test), callbacks=callbacks_list)
    return model, history

In [16]:
#Create and Fit Model
model = createModel(64, 4)
model, history = run_model(model, x_train, y_train, x_test, y_test, batch_size=512, epochs=50)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 64)                93440     
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 260       
Total params: 93,700
Trainable params: 93,700
Non-trainable params: 0
_________________________________________________________________
Train on 22979 samples, validate on 9616 samples
Epoch 1/50
22979/22979 [==============================] - 58s 3ms/step - loss: 0.1991 - acc: 0.2824 - val_loss: 0.1841 - val_acc: 0.3001
Epoch 2/50
22979/22979 [==============================] - 51s 2ms/step - loss: 0.1829 - acc: 0.3059 - val_loss: 0.1819 - val_acc: 0.3143
Epoch 3/50
22979/22979 [==============================] - 51s 2ms/step - loss: 0.1519 - acc: 0.4625 - val_loss: 0.1370 - val_acc: 0.5203
Epoch 4/50
22979/22979 [==============================] - 50s 2ms/step - l

Epoch 50/50
22979/22979 [==============================] - 51s 2ms/step - loss: 0.0323 - acc: 0.9095 - val_loss: 0.0493 - val_acc: 0.8637

Epoch 00050: val_acc did not improve
